In [13]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
# from langchain.output_parsers import ResponseSchema,StructuredOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv()


True

# Output Parsers in LangChain 
        string output parser
        pydantic output parser
        jason output parser
        csv output parser
        structured output parser

In [2]:
# llm=HuggingFaceEndpoint(
#     repo_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
#     task="text-generation",
#     # temperature= 0.7
#     )

# chat=ChatHuggingFace(llm=llm)
# print(chat.invoke(["What is the capital of France?"]))

In [3]:
llm=HuggingFaceEndpoint(repo_id="deepseek-ai/DeepSeek-V3.2",
                        temperature= 1.5
                        ,task="text-generation")

chat=ChatHuggingFace(llm=llm,)
print(chat.invoke("What is the capital of France?"))

d:\AI_agent_repo\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


content='The capital of France is Paris.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25}, 'model_name': 'deepseek-ai/DeepSeek-V3.2', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c7829-e94a-72f2-97f3-450756771617-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 17, 'output_tokens': 8, 'total_tokens': 25}


In [5]:
# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

parser = StrOutputParser()

chain = template1 | chat | parser | template2 | chat | parser

result = chain.invoke({'topic':'black hole'})

print(result)


Black holes are extreme cosmic objects with powerful gravity, formed from collapsed massive stars. They come in types like stellar, supermassive, and intermediate-mass. Advances like the Event Horizon Telescope now make black holes observable.


In [9]:
parser = JsonOutputParser()

template = PromptTemplate(
    template='Give me 5 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = template | chat | parser

result = chain.invoke({'topic':'black hole'})

print(result)

{'facts_about_black_holes': [{'fact': "The boundary of a black hole is called the event horizon. Once anything, including light, crosses this point, it cannot escape the black hole's gravitational pull."}, {'fact': "Black holes can vary in size. Stellar black holes form from collapsed stars and can be several times the Sun's mass, while supermassive black holes, found at the centers of galaxies, can contain millions or billions of solar masses."}, {'fact': "Despite their name, black holes are not entirely 'black' due to Hawking radiation—a theoretical process where black holes can emit particles and lose mass over extremely long timescales."}, {'fact': "Black holes distort time and space around them, an effect predicted by Einstein's theory of general relativity known as gravitational time dilation."}, {'fact': "The first-ever image of a black hole's event horizon, captured by the Event Horizon Telescope in 2019, shows the supermassive black hole at the center of the galaxy M87."}]}


In [ ]:
model=chat
schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'black hole'})

print(result)

In [15]:
model=chat
class Person(BaseModel):

    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

final_result = chain.invoke({'place':'sri lankan'})

print(final_result)

name='Asiri Perera' age=25 city='Colombo'
